In [ ]:
# graph 문제를 풀기 위한 torch, compuet vision문제를 보다 쉽게 해결할 수 있는 모듈인 torchvision import
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose

In [ ]:
def set_manual_seed(seed=42):
  """
    난수 발생시 고정된 난수를 발생시키기 위해 python local package
    random, numpy (다차원의 메트릭을 다루기위환 package), pytorch
    seed를 고정하는 function 입니다.
    난수 발생시 나오는 문제를 피하기 위해 미리 seed를 고정시켜놓는다. 
  """
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
set_manual_seed()

In [ ]:
"""
  학습을 위한 데이터 다운로드 및 Object Class형태로 변환 
  학습 데이터 : 60000개
"""
train_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)
"""
  평가 위한 데이터 다운로드 및 Object Class형태로 변환 
  평가 데이터 : 10000개
"""
validation_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
"""
  1. 학습 모델에 필요한 HyperParameters
  2. GPU 혹은 CPU를 사용하여 학습
  전체 데이터에서 몇개의 데이터를 구분지을 것인가
""" 
batch_size = 128
epochs = 10
learning_rate = 1e-3
log_step = 100
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
"""
  모델 학습과 평가를 위한 데이터 로더
"""
train_loader = DataLoader(train_data, batch_size=batch_size)  #batch size는 전체 데이터 사이즈에서 나눈다 0에 수렴하는가를 확인해본다.
validation_loader = DataLoader(validation_data, batch_size=batch_size)

In [ ]:
# neural network
class NeuralNet(nn.Module):
    
    def __init__(self):
        super(NeuralNet, self).__init__()
        # image shape or size: [1, 28, 28] Number of batch, Channel size (3,1), H: 28, W: 28
        """
        1. 최초에 입력받을 이미지의 shape가 [1, 28 ,28] 이기 때문에 1차원 배열로 변경 28 * 28 * 1 그렇기 위해 feature_flatten이 필요
        + 항상 레이어의 outpu_features 갯수와 다음 레이어의 input_features의 수가 동일해야함
        """
        self.feature_flatten = nn.Flatten()
        """
        2. Fully Connected Layer + ReLU(actiation)을 이용하여 첫번째 레이어를 형성함
        """
        self.linear_1 = nn.Linear(in_features=28 * 28, out_features=512)
        self.relu_1 = nn.ReLU()
        """
        3. Fully Connected Layer + ReLU(actiation)을 이용하여 두번째 레이어를 형성함
        """
        self.linear_2 = nn.Linear(in_features=512, out_features=512) 
        self.relu_2 = nn.ReLU()
        """
        4. Fully Connected Layer + ReLU(actiation)을 이용하여 마지막 레이어를 형성함
        """
        self.linear_3 = nn.Linear(in_features=512, out_features=10)
        self.relu_3 = nn.ReLU()
    
    def forward(self, x):
        """
            :x pram: input image tensor (batch, channel, height, widht)
        """
        x = self.feature_flatten(x)
        x = self.relu_1(self.linear_1(x))
        x = self.relu_2(self.linear_2(x))
        logits = self.relu_3(self.linear_3(x))

        return logits
        

In [ ]:
"""
  모델이 학습이 잘되고 있는가를 평가할 수 있는 함수인 loss function 정의
  loss function이 0에 수렴할 수 있도록 최적화 알고리즘인 Gradient Decent 정의
"""
model = NeuralNet().to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def train_op(train_loader, model, loss_fn, optim):
    data_size = len(train_loader.dataset)
    model.train()
    for batch, (input_images, input_labels) in enumerate(train_loader):
        """
            traing data loader에서 이미지와 레이블을 가지고와 GPU연산이 가능 하도록 injection
        """
        input_images = input_images.to(device)
        input_labels = input_labels.to(device)
        
        prediction = model(input_images)
        """
            예측한 결과와 실제로 정답지를 비교해가면서 loss의 값을 축적함
        """
        loss = loss_fn(prediction, input_labels)
        """
            마지막에 정답이라고 말한 y_hat의 값을 역전파하여 파라미터를 업데이트함
            -> e.g. 티셔츠를 부추라고 예측하였으면 다시 티셔츠라고 알려주는 행위
        """
        optim.zero_grad()
        loss.backward()
        optim.step()
        """
            학습시에 모델의 loss를 모니터링 위한 로그
        """
        if batch % log_step == 0:
            print(f'loss function value: {loss.item():.5f}')

In [ ]:
def test_op(validation_loader, model, loss_fn):
  data_size = len(validation_loader.dataset)
  batches = len(validation_loader)
  model.eval()  #test를 할 때 사용한다. 이류를 찾아보자
  test_loss, correct = 0,0
  with torch.no_grad(): #미분을 수행하지 않겠다. 
    for input_test_images, input_test_labels in test_loader:
      #cpu->Gpu
      input_test_images,input_test_labels = input_test_images.to(device), input_test_labels.to(device)

      #compute error and prediction
      prediction = model(input_test_images)    
      test_loss += loss_fn(prediction, input_test_labels).item()
      correct += (prediction.argmax(1) == input_test_labels).type(torch.float32).sum().item()

    test_loss /= batches
    accuarcy = correct / data_size
    print(f'test accuracy: {accuarcy:.4f}')

In [ ]:
for _ in range(epochs):
  train_op(train_loader, model, loss_fn, optim)
  """
    step : test operation
  """

In [ ]:
torch.save(model.state_dict(),'latest.pth')

In [ ]:
del model

In [ ]:
model = NeuralNet()
model.load_state_dict(torch.load('latest.pth',map_location = device))

In [ ]:
#시각화 & 모델 예측
import matplotlib.pyplot as plt
classes = [
    "shirt",
    "trouser",
    "pullover",
    "dress",
    "coat",
    "sandal",
    "shirt",
    "sneaker",
    "bag",
    "ankle boot",
]

model.eval()  
rand_idx = random.randint(0,len(validation_data)-1)

input_image,input_label= validation_data[rand_idx][0],validation_data[rand_idx][1]
fig = plt.figure(figsize = (8,8))
with torch.no_grad():
  print(f'current data index : {rand_idx}')
  y_hat = model(input_image)
  prediction = classes[torch.argmax(y_hat.squeeze())]
  decode_label = classes[input_label]
  print(f'prediction result:{prediction}, actually:{decode_label}')
  plt.imshow(input_image.squeeze())